In [ ]:
import geemap
import ee
from Monarch import user_gee

In [ ]:
Map = geemap.Map()
Map

In [ ]:
roi = geemap.geojson_to_ee(r'H:/Monarch/paper/Data/area.geojson').geometry()
Map.center_object(roi, 12)
Map.addLayer(roi, {}, 'roi')

In [ ]:
visParams = {'max': 3000, 'min': 0, 'bands': ['B4', 'B3', 'B2']}

In [ ]:
for year in range(2018, 2019):
    s2_year = []
    for i in range(3, 12):
        s2 = ee.ImageCollection("COPERNICUS/S2")\
                .filterBounds(roi)\
                .sort('CLOUD_COVERAGE_ASSESSMENT', False)\
                .filterDate(str(year), str(year+1))\
                .filter(ee.Filter.calendarRange(i, i, 'month'))\
                .map(user_gee.rm_cloud_s2_sr)
        a = s2.aggregate_array('system:time_start').map(lambda i: ee.Date(i).format('YYYY-MM-dd'))
        s2_year.append(s2.mosaic().clip(roi).set('time', a.get(s2.size().subtract(1))))
    s2_year = ee.ImageCollection(s2_year)
    geemap.download_ee_video(s2_year, visParams, f'H:/Monarch/paper/Data/S2_image_Thumbnail_git/S2_{year}.gif')

In [ ]:
geemap.add_text_to_gif(f'H:/Monarch/paper/Data/S2_image_Thumbnail_git/S2_{year}.gif', f'H:/Monarch/paper/Data/S2_image_Thumbnail_git/S2_{year}_3.gif', xy=('3%', '5%'), text_sequence=a, 
                       font_size=20, font_color='#ffffff', duration=1000)

In [ ]:
a = s2_year.aggregate_array('time').getInfo()

In [ ]:
a